In [1]:
import re
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
import cleanlab
from cleanlab.classification import CleanLearning
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd

In [3]:
train_df = pd.read_csv('/data/ephemeral/data/aug_data_특수문자제거.csv')
test_data = pd.read_csv("/data/ephemeral/code/train_df3.csv")

In [4]:
train_df['ratio'] = None

for idx, row in train_df.iterrows():
    text = row['text']
    kor_cnt = len(re.findall(r"[가-힣ㄱ-ㅎ]", text))
    blank_cnt = len(re.findall(r" ", text))

    ratio = kor_cnt / (len(text) - blank_cnt)
    train_df.loc[idx, 'ratio'] = ratio

In [5]:
train70_df = train_df[(train_df['ratio'].between(0.6, 0.65, inclusive='both'))].drop('ratio', axis=1)
train70_df.shape

(196, 3)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

MODEL_NAME = "/data/ephemeral/home/.cache/huggingface/hub/models--DeepMount00--Llama-3.1-8b-Ita/snapshots/2fb3019939a71ff8986ea8ef4bb1bddd1c39baf3"

model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.bfloat16).eval()
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1
...,...,...,...
1090,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1091,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1092,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
1093,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [198]:
test_data

,ID,text,target
0,ynat-v1_train_00000,"KT, SKT, LGU+, 3대 통신사 이용료 인상... 정기요금제로 전환, 이용자...",4
1,ynat-v1_train_00001,대한민국 최초의 여성 국회의장으로 선출된 이명박 전 대통령의 딸 이희수 국회의원이 ...,3
2,ynat-v1_train_00002,"북한, 남북통일을 위한 자주통일의 길을 열어야 한다",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,미국 대선 앞두고 선거법 위,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
2797,ynat-v1_train_02797,"텔레그램, 한드 등 아시아서 2시간 동안 서비스 중단... 카카오T, 구글 메신저도...",4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [200]:
import pandas as pd
import numpy as np
import wandb
from cleanlab.classification import CleanLearning
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [199]:
model_name = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(train_data['target'].unique()))

# 토큰화 함수
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", max_length=64)

train_dataset = Dataset.from_pandas(train_data)
test_dataset = Dataset.from_pandas(test_data)
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
train_dataset = train_dataset.rename_column("target", "labels")
test_dataset = test_dataset.rename_column("target", "labels")
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=16,
    logging_strategy="steps",       # 로그를 기록할 빈도 설정
    logging_steps=10,
    num_train_epochs=5,
    weight_decay=0.01,
    report_to="wandb",              # wandb에 로그 기록
    run_name="bert-finetuning"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

trainer.train()
wandb.finish()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1705 [00:00<?, ? examples/s]

Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss
1,1.310400,1.715550
2,1.103700,1.619121
3,0.883300,1.545385
4,0.724700,1.562865
5,0.431500,1.554673


NameError: name 'wandb' is not defined

In [201]:
test_data_unique = test_data[~test_data["text"].isin(train_data["text"])].reset_index(drop=True)
test_data_unique

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1
...,...,...,...
1090,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1091,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1092,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
1093,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [178]:
test_data_unique

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,0
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,6
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,1
...,...,...,...
1090,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1091,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1092,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2
1093,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [202]:
from tqdm import tqdm
from torch.utils.data import DataLoader
from cleanlab.filter import find_label_issues

model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

test_dataset = Dataset.from_pandas(test_data_unique)
test_dataset = test_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.rename_column("target", "labels")
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataloader = DataLoader(test_dataset, batch_size=8)

pred_probs = []
for batch in tqdm(test_dataloader):
    # 입력 데이터를 GPU로 이동
    batch = {k: v.to(device) for k, v in batch.items() if k in ["input_ids", "attention_mask"]}
    
    with torch.no_grad():
        outputs = model(**batch)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=1).cpu().numpy()  # CPU로 변환하여 numpy 배열로 저장
    pred_probs.extend(probs)  # 각 배치의 예측 확률을 리스트에 추가

Map:   0%|          | 0/1095 [00:00<?, ? examples/s]

100%|██████████| 137/137 [00:07<00:00, 19.18it/s]


In [211]:
max_probs = np.max(pred_probs, axis=1)
predicted_labels = np.argmax(pred_probs, axis=1)

# 원본 데이터 복사 (또는 수정할 새로운 DataFrame 생성)
updated_data = test_data_unique.copy()

# 레이블 수정
for i in range(len(updated_data)):
    original_label = updated_data.loc[i, "target"]  # 원래 레이블
    if max_probs[i] >= 0.3 and predicted_labels[i] != original_label:
        # 예측 확률이 0.3 이상이고, 원래 레이블과 다르면 수정
        updated_data.loc[i, "target"] = predicted_labels[i]


In [212]:
updated_data

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6
2,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2
3,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4
4,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5
...,...,...,...
1090,ynat-v1_train_02794,문 대통령 김기식 금감원장 사표 수리키로종합,2
1091,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
1092,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4
1093,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [213]:
# test_data의 인덱스를 기준으로 업데이트
for i, row in updated_data.iterrows():
    # test_data에서 동일한 텍스트를 가진 행의 인덱스 찾기
    index_in_test_data = test_data[test_data["text"] == row["text"]].index
    # 레이블 업데이트
    if len(index_in_test_data) > 0:  # 같은 텍스트가 있는 경우
        test_data.loc[index_in_test_data, "target"] = row["target"]

test_data

,ID,text,target
0,ynat-v1_train_00000,"KT, SKT, LGU+, 3대 통신사 이용료 인상... 정기요금제로 전환, 이용자...",4
1,ynat-v1_train_00001,대한민국 최초의 여성 국회의장으로 선출된 이명박 전 대통령의 딸 이희수 국회의원이 ...,3
2,ynat-v1_train_00002,"북한, 남북통일을 위한 자주통일의 길을 열어야 한다",2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
4,ynat-v1_train_00004,미국 대선 앞두고 선거법 위,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4
2797,ynat-v1_train_02797,"텔레그램, 한드 등 아시아서 2시간 동안 서비스 중단... 카카오T, 구글 메신저도...",4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1


In [215]:
test_data.to_csv("roberta_large_relabel.csv", index=False, encoding="utf-8-sig")

In [214]:
test_data['target'].value_counts()

target
0    422
3    416
6    409
2    406
1    394
4    392
5    361
Name: count, dtype: int64

In [193]:
# Cleanlab으로 레이블 오류 탐지
pred_probs = np.array(pred_probs)  # 리스트를 numpy 배열로 변환
label_issues = find_label_issues(
    labels=np.array(test_data_unique["labels"]),  # test_dataset의 정답 라벨
    pred_probs=pred_probs,
    return_indices_ranked_by="self_confidence"
)

# 레이블 오류가 의심되는 샘플 인덱스 출력
print("라벨 오류가 의심되는 샘플 인덱스:", label_issues)

KeyError: 'labels'

1095

In [169]:
test_dataset_unique

Dataset({
    features: ['ID', 'text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 2800
})

0: 생활문화
1: 스포츠
2: 정치
3: 사회
4: IT과학
5: 경제
6: 세계

In [103]:
data['target'] = label_issues['predicted_label']

In [101]:
# CSV 파일로 저장
data.to_csv("data_relabel_removeno.csv", index=False, encoding='utf-8-sig')